In [1]:
import pandas as pd
import json
def prettyjson(final_dict):
    print(json.dumps(final_dict, ensure_ascii=False, indent=4))

df_nurse_record = pd.read_csv("data_processing/NurseRecord_full.csv")

/var/folders/pj/6yc3mtz11t5bfkf7bljn52z40000gn/T/ipykernel_8847/2924052718.py:6: DtypeWarning: Columns (9,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nurse_record = pd.read_csv("data_processing/NurseRecord_full.csv")


In [2]:
unique_numbers_list = df_nurse_record['연구등록번호'].unique().tolist()
unique_numbers_list

[11729736,
 11719538,
 11719074,
 11716267,
 11713362,
 11719073,
 11711062,
 11708731,
 11701700,
 11698881,
 11693779,
 11688056,
 11691445,
 11687877,
 11685481,
 11679744,
 11678265,
 11680244,
 11684395,
 11671419,
 11670092,
 11673730,
 11673458,
 11681908,
 11671021,
 11670943,
 11664210,
 11664098,
 11663074,
 11669071,
 11666364,
 11662505,
 11662039,
 11658927,
 11658802,
 11675015,
 11657394,
 11656921,
 11655581,
 11657563,
 11651611,
 11642146,
 11655558,
 11640636,
 11655355,
 11621077,
 11634023,
 11646163,
 11618341,
 11616818,
 11621258,
 11612316,
 11608510,
 11599335,
 11604764,
 11604245,
 11602568,
 11594026,
 11606703,
 11593096,
 11654315,
 11593016,
 11590072,
 11588174,
 11586192,
 11585050,
 11586160,
 11581935,
 11581512,
 11578830,
 11581406,
 11575878,
 11580145,
 11580142,
 11575716,
 11572411,
 11569843,
 11563213,
 11552695,
 11543033,
 11559506,
 11562210,
 11519388,
 11527166,
 11524055,
 11518327,
 11542837,
 11506768,
 11479540,
 11461522,
 11483475,

In [3]:
import os
# [int(i[:-5]) for i in os.listdir("TestData")]
list(set(unique_numbers_list) & set([int(i[:-5]) for i in os.listdir("TestData")]))

[11439744,
 11079810,
 9927172,
 11510026,
 10919821,
 11196686,
 11387921,
 11411731,
 11376409,
 10532891,
 2666140,
 11651611,
 11428510,
 11059230,
 10810106,
 2679585,
 9546914,
 3114273,
 11470115,
 4737827,
 11571617,
 11176999,
 11473575,
 11421863,
 10989867,
 10445879,
 11552695,
 11591481,
 11580218,
 11519388,
 3526716,
 11505853,
 10542010,
 2319806,
 11133121,
 11588546,
 4394434,
 11701700,
 4363848,
 10820683,
 1674060,
 10816078,
 11203534,
 4849615,
 11452367,
 2384467,
 11646163,
 11693779,
 3756372,
 11519445,
 11396441,
 11657563,
 11581406,
 2395999,
 1708385,
 10391780,
 10216420,
 11419879,
 10435818,
 4526187,
 11431914,
 4581103,
 11163376,
 11410288,
 11356658,
 11680244,
 11452414,
 11449976,
 11543033,
 11417722,
 11671419,
 11527166,
 4567551]

In [4]:
def get_nursing_record(patient_id: int):
    patient_data = df_nurse_record[df_nurse_record['연구등록번호'] == patient_id]
    columns = ['간호중재_코드명', '간호활동_코드명', '간호속성코드_코드명', '속성', 'Duty_코드명', '시행일시', '간호진단_프로토콜_코드명']
    result = patient_data[columns].sort_values('시행일시')

    # 시행일시별로 그룹화
    final_dict = {}
    grouped = result.groupby('시행일시')

    for time_key, group in grouped:
        # 각 그룹의 데이터를 리스트로 변환
        time_data = []
        for index, row in group.iterrows():
            time_data.append({
                "시행일시": row['시행일시'],
                "간호진단프로토콜(코드명)": row['간호진단_프로토콜_코드명'],
                "간호중재(코드명)": row['간호중재_코드명'],
                "간호활동(코드명)": row['간호활동_코드명'], 
                "간호속성코드(코드명)": row['간호속성코드_코드명'],
                "속성": row['속성'],
                "Duty(코드명)": row['Duty_코드명'],
            })
        
        final_dict[time_key] = time_data
    return final_dict

In [5]:
# 실제 입퇴원 기간을 몰라. 그러니 해당 기간에 처음 울린 알람과 나중에 울린 알람
def get_admission_periods(patient_id):
    r = json.loads(open(f'TestData/{patient_id}.json', 'r', encoding='utf-8').read())
    admission_periods = []
    for idx, i in enumerate(sorted(list(r['DatSeq'].keys()), key=int)):
        alarms = list(r['DatSeq'][i].keys())
        admission_periods.append({
            "start": alarms[0].split(' ')[0],
            "end": alarms[-1].split(' ')[0],
            "id": f"admission{idx+1}"
        })
    return admission_periods

In [ ]:
# TimeStamp
# PatSeq
# SequenceNumber
# AlertId
# Source
# Label
# Severity
# Kind
# AnnounceTime
# AlertTime
# II_wave
# II_time_diff_sec
# ABP_wave
# ABP_time_diff_sec
# Pleth_wave
# Pleth_time_diff_sec
# Resp_wave
# Resp_time_diff_sec
# SpO2_numeric
# SpO2_numeric_time_diff_sec
# Pulse_numeric
# Pulse_numeric_time_diff_sec
# ST_numeric
# ST_time_diff_sec
# Tskin_numeric
# Tskin_numeric_time_diff_sec
# ABP_numeric
# ABP_numeric_time_diff_sec
# NBP_numeric
# NBP_numeric_time_diff_sec
# Perf_numeric
# Perf_numeric_time_diff_sec
# PPV_numeric
# PPV_numeric_time_diff_sec
# AlertRow
# ClosestNursing
# Nursing_TimeDiffSec

In [11]:
def get_alarms_and_waveforms(patient_id):
    r = json.loads(open(f'TestData/{patient_id}.json', 'r', encoding='utf-8').read())
    waveforms = {}
    alarms = {}

    for idx, i in enumerate(sorted(list(r['DatSeq'].keys()), key=int)):
        for ts in list(r['DatSeq'][i].keys()):

            
            # 2024-01-09 15:32:20.657
            data = r['DatSeq'][i][ts][0]

            # 2024-01-09
            date = ts.split(' ')[0]

            # 15:32:20
            hms = ts.split(' ')[-1].split('.')[0]
            
            # SpO2_numeric
            # SpO2_numeric_time_diff_sec
            # Pulse_numeric
            # Pulse_numeric_time_diff_sec
            # ST_numeric
            # ST_time_diff_sec
            # Tskin_numeric
            # Tskin_numeric_time_diff_sec
            # ABP_numeric
            # ABP_numeric_time_diff_sec
            # NBP_numeric
            # NBP_numeric_time_diff_sec
            # Perf_numeric
            # Perf_numeric_time_diff_sec
            # PPV_numeric
            # PPV_numeric_time_diff_sec

            numeric_dict = {
                data['SpO2_numeric']['Label']: data['SpO2_numeric']['Value'],
                data['Pulse_numeric']['Label']: data['Pulse_numeric']['Value'],
                data['ST_numeric']['Label']: data['ST_numeric']['Value'],
                data['Tskin_numeric']['Label']: data['Tskin_numeric']['Value'],
                data['ABP_numeric']['Label']: data['ABP_numeric']['Value'],
                data['NBP_numeric']['Label']: data['NBP_numeric']['Value'],
                data['Perf_numeric']['Label']: data['Perf_numeric']['Value'],
                data['PPV_numeric']['Label']: data['PPV_numeric']['Value'],
            }
            # print(numeric_dict)

            Severity = data["Severity"][0]
            Severity_color = {0: "Red", 1:"Yellow", 2:"ShortYellow", 3:"SevereCyan", 4:"Cyan", 5:"SilentCyan", 6:"White"}[Severity]

            II_time_diff_sec = data["II_time_diff_sec"]
            II_wave = data["II_wave"]["WaveSamples_Base64_cal"]

            ABP_time_diff_sec = data["ABP_time_diff_sec"]
            ABP_wave = data["ABP_wave"]["WaveSamples_Base64_cal"]

            Resp_time_diff_sec = data["Resp_time_diff_sec"]
            Resp_wave = data["Resp_wave"]["WaveSamples_Base64_cal"]

            Pleth_time_diff_sec = data["Pleth_time_diff_sec"]
            Pleth_wave = data["Pleth_wave"]["WaveSamples_Base64_cal"]


            if not f'admission{idx+1}' in alarms.keys():
                alarms[f'admission{idx+1}'] = {}
            if not date in alarms[f'admission{idx+1}'].keys():
                alarms[f'admission{idx+1}'][date] = []


            # 이거 하기 전에 조건 거는걸로 하자.
            alarms[f'admission{idx+1}'][date].append({
                "time": hms,
                "color": Severity_color,
                "id": f"{patient_id}-{date}-{hms}",
                "timestamp": f"{date} {hms}"
            })
            # 이거 하기 전에 조건 거는걸로 하자.
            waveforms[f"{date} {hms}"] = {
                "ABP": ABP_wave,
                "Lead-II": II_wave,
                "Resp": Resp_wave,
                "Pleth": Pleth_wave,
                "Numeric": numeric_dict
            }
            # break # 하나만 해서 넣고싶을 때 주석 풀면 됨.

    return alarms, waveforms
                


In [12]:
import os
for patient_id in list(set(unique_numbers_list) & set([int(i[:-5]) for i in os.listdir("TestData")])):
    try:
        # patient_id = 10216420
        admissions = get_admission_periods(patient_id)
        nursing_records = get_nursing_record(patient_id)
        alarms, waveforms = get_alarms_and_waveforms(patient_id)
    except:
        continue


    DataConverter = {
        "admission_periods": admissions,
        "alarms": alarms,
        "nursing_records": nursing_records,
        "waveforms": waveforms
    }

    if not nursing_records:
        print(patient_id)

    with open(f'DATA/{patient_id}.json', 'w', encoding='utf-8') as f:
        json.dump(DataConverter, f, ensure_ascii=False, indent=4)

    print(f"DATA/{patient_id}.json 파일 저장 완료!")
    # break
# import os
# ids = [i[:-5] for i in os.listdir('TestData')]

DATA/11439744.json 파일 저장 완료!
DATA/11079810.json 파일 저장 완료!
DATA/11510026.json 파일 저장 완료!
DATA/11196686.json 파일 저장 완료!
DATA/11376409.json 파일 저장 완료!
DATA/2666140.json 파일 저장 완료!
DATA/11651611.json 파일 저장 완료!
DATA/11059230.json 파일 저장 완료!
DATA/10810106.json 파일 저장 완료!
DATA/2679585.json 파일 저장 완료!
DATA/3114273.json 파일 저장 완료!
DATA/11470115.json 파일 저장 완료!
DATA/11176999.json 파일 저장 완료!
DATA/11473575.json 파일 저장 완료!
DATA/11421863.json 파일 저장 완료!
DATA/11591481.json 파일 저장 완료!
DATA/11580218.json 파일 저장 완료!
DATA/11519388.json 파일 저장 완료!
DATA/11133121.json 파일 저장 완료!
DATA/4394434.json 파일 저장 완료!
DATA/11701700.json 파일 저장 완료!
DATA/10820683.json 파일 저장 완료!
DATA/1674060.json 파일 저장 완료!
DATA/10816078.json 파일 저장 완료!
DATA/11203534.json 파일 저장 완료!
DATA/4849615.json 파일 저장 완료!
DATA/11452367.json 파일 저장 완료!
DATA/2384467.json 파일 저장 완료!
DATA/11646163.json 파일 저장 완료!
DATA/11693779.json 파일 저장 완료!
DATA/3756372.json 파일 저장 완료!
DATA/11396441.json 파일 저장 완료!
DATA/11657563.json 파일 저장 완료!
DATA/2395999.json 파일 저장 완료!
DATA/1708385.json 파일 저장